In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.utils.np_utils import to_categorical
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam

train_df = pd.read_json('../input/train.json')
test_df = pd.read_json('../input/test.json')

Using TensorFlow backend.


In [2]:
def get_image(df):
    '''Create 3-channel 'images'. Return rescale-normalised images.'''
    images = []
    for i, row in df.iterrows():
        # Formulate the bands as 75x75 arrays
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = (band_1 + band_2)/2

        # Rescale
        r = (band_1 - band_1.min()) / (band_1.max() - band_1.min())
        g = (band_2 - band_2.min()) / (band_2.max() - band_2.min())
        b = (band_3 - band_3.min()) / (band_3.max() - band_3.min())

        rgb = np.dstack((r, g, b))
        images.append(rgb)
    return np.array(images)


train_x = get_image(train_df)
test_x = get_image(test_df)

print(train_x.shape,test_x.shape)

(1604, 75, 75, 3) (8424, 75, 75, 3)


In [3]:
y = train_df.is_iceberg.values
print(y[:5])
from keras.callbacks import ModelCheckpoint,LearningRateScheduler
from keras.layers import Input
from keras.models import Model
from sklearn.model_selection import KFold
from keras.preprocessing.image import ImageDataGenerator
import pickle
from sklearn.metrics import log_loss

def lr_f(epoch):
    if epoch < 15:
        return 0.0005
    elif epoch < 30:
        return 0.0001
    elif epoch < 45:
        return 0.00005
    else:
        return 0.00001

[0 0 1 0 0]


In [8]:
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.layers import Input,AveragePooling2D,GlobalMaxPooling2D,GlobalAveragePooling2D,Flatten
from keras import layers
from keras.models import Model

def conv2d_bn(x,
              filters,
              num_row,
              num_col,
              padding='same',
              strides=(1, 1),
              name=None):

    if name is not None:
        bn_name = name + '_bn'
        conv_name = name + '_conv'
    else:
        bn_name = None
        conv_name = None
    bn_axis = 3
    
    x = Conv2D(
        filters, (num_row, num_col),
        strides=strides,
        padding=padding,
        use_bias=False,
        name=conv_name)(x)
    x = BatchNormalization(axis=bn_axis, scale=False, name=bn_name)(x)
    x = Activation('relu', name=name)(x)
    return x

def create_incept_model():
    img_input = Input(shape=(75,75,3))
    channel_axis = 3
    
    # bn
    x = conv2d_bn(img_input, 32, 3, 3, strides=(2, 2), padding='valid')
    x = ZeroPadding2D((1, 1))(x)
    x = conv2d_bn(x, 32, 3, 3, padding='valid')
    x = conv2d_bn(x, 64, 3, 3)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)
    
    x = ZeroPadding2D((1, 1))(x)
    x = conv2d_bn(x, 64, 1, 1, padding='valid')
    x = conv2d_bn(x, 64, 3, 3, padding='valid')
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

     # mixed 0, 1, 2: 35 x 35 x 256
    branch1x1 = conv2d_bn(x, 64, 1, 1)

    branch5x5 = conv2d_bn(x, 48, 1, 1)
    branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 32, 1, 1)
    x = layers.concatenate(
        [branch1x1, branch5x5, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed0')
    x = ZeroPadding2D((1, 1))(x)

    # mixed 1: 35 x 35 x 256
    branch1x1 = conv2d_bn(x, 64, 1, 1)

    branch5x5 = conv2d_bn(x, 48, 1, 1)
    branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 64, 1, 1)
    x = layers.concatenate(
        [branch1x1, branch5x5, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed1')
    x = ZeroPadding2D((1, 1))(x)

    # mixed 2: 35 x 35 x 256
    branch1x1 = conv2d_bn(x, 64, 1, 1)

    branch5x5 = conv2d_bn(x, 48, 1, 1)
    branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 64, 1, 1)
    x = layers.concatenate(
        [branch1x1, branch5x5, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed2')
    x = ZeroPadding2D((1, 1))(x)
    
    # mixed 3: 17 x 17 x 768
    branch3x3 = conv2d_bn(x, 384, 3, 3, strides=(2, 2), padding='valid')

    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(
        branch3x3dbl, 96, 3, 3, strides=(2, 2), padding='valid')

    branch_pool = MaxPooling2D((3, 3), strides=(2, 2))(x)
    x = layers.concatenate(
        [branch3x3, branch3x3dbl, branch_pool], axis=channel_axis, name='mixed3')

    x = Conv2D(96, 3, strides=2, padding='same',activation='relu')(x)
    x = Flatten()(x)
    x = Dense(256,activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(1,activation='sigmoid')(x)
    
    return Model(img_input, x, name='inception_v3')
print('model model')

model model


In [9]:
def kfold_train(fold_cnt=3,rnd=99):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_incept_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        early_stopping = EarlyStopping(monitor = 'val_loss', patience = 20, verbose = 0, mode= 'min')
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=120, 
                  verbose=2,
                  callbacks=[model_chk,lr_s,early_stopping]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=5)

Epoch 1/120
Epoch 00001: val_loss improved from inf to 0.69615, saving model to best_m.h5
 - 12s - loss: 0.6647 - acc: 0.6718 - val_loss: 0.6961 - val_acc: 0.5389
Epoch 2/120
Epoch 00002: val_loss did not improve
 - 6s - loss: 0.5169 - acc: 0.7476 - val_loss: 0.8253 - val_acc: 0.5389
Epoch 3/120
Epoch 00003: val_loss did not improve
 - 6s - loss: 0.4728 - acc: 0.7841 - val_loss: 0.7864 - val_acc: 0.5389
Epoch 4/120
Epoch 00004: val_loss did not improve
 - 6s - loss: 0.4538 - acc: 0.7929 - val_loss: 0.9291 - val_acc: 0.5389
Epoch 5/120
Epoch 00005: val_loss did not improve
 - 6s - loss: 0.3715 - acc: 0.8334 - val_loss: 0.9528 - val_acc: 0.5452
Epoch 6/120
Epoch 00006: val_loss improved from 0.69615 to 0.64626, saving model to best_m.h5
 - 6s - loss: 0.3742 - acc: 0.8052 - val_loss: 0.6463 - val_acc: 0.7352
Epoch 7/120
Epoch 00007: val_loss improved from 0.64626 to 0.50936, saving model to best_m.h5
 - 6s - loss: 0.3565 - acc: 0.8427 - val_loss: 0.5094 - val_acc: 0.7695
Epoch 8/120
Epoch

Epoch 63/120
Epoch 00063: val_loss did not improve
 - 6s - loss: 0.2035 - acc: 0.9162 - val_loss: 0.2495 - val_acc: 0.8754
Epoch 64/120
Epoch 00064: val_loss did not improve
 - 6s - loss: 0.2087 - acc: 0.9062 - val_loss: 0.2436 - val_acc: 0.8816
Epoch 65/120
Epoch 00065: val_loss did not improve
 - 6s - loss: 0.2056 - acc: 0.9234 - val_loss: 0.2457 - val_acc: 0.8847
Epoch 66/120
Epoch 00066: val_loss did not improve
 - 6s - loss: 0.2223 - acc: 0.9035 - val_loss: 0.2427 - val_acc: 0.8847
Epoch 67/120
Epoch 00067: val_loss did not improve
 - 6s - loss: 0.1973 - acc: 0.9162 - val_loss: 0.2440 - val_acc: 0.8879
Epoch 68/120
Epoch 00068: val_loss did not improve
 - 6s - loss: 0.2020 - acc: 0.9201 - val_loss: 0.2404 - val_acc: 0.8785
Epoch 69/120
Epoch 00069: val_loss did not improve
 - 6s - loss: 0.2148 - acc: 0.9058 - val_loss: 0.2465 - val_acc: 0.8847
Epoch 70/120
Epoch 00070: val_loss improved from 0.23986 to 0.23941, saving model to best_m.h5
 - 6s - loss: 0.2205 - acc: 0.9125 - val_los

Epoch 5/120
Epoch 00005: val_loss improved from 0.68655 to 0.67931, saving model to best_m.h5
 - 6s - loss: 0.3866 - acc: 0.8234 - val_loss: 0.6793 - val_acc: 0.5639
Epoch 6/120
Epoch 00006: val_loss did not improve
 - 6s - loss: 0.3735 - acc: 0.8259 - val_loss: 0.9769 - val_acc: 0.6511
Epoch 7/120
Epoch 00007: val_loss improved from 0.67931 to 0.43408, saving model to best_m.h5
 - 6s - loss: 0.3747 - acc: 0.8183 - val_loss: 0.4341 - val_acc: 0.7788
Epoch 8/120
Epoch 00008: val_loss did not improve
 - 6s - loss: 0.4055 - acc: 0.8201 - val_loss: 0.4649 - val_acc: 0.7165
Epoch 9/120
Epoch 00009: val_loss improved from 0.43408 to 0.36334, saving model to best_m.h5
 - 6s - loss: 0.3640 - acc: 0.8390 - val_loss: 0.3633 - val_acc: 0.7944
Epoch 10/120
Epoch 00010: val_loss did not improve
 - 7s - loss: 0.3650 - acc: 0.8175 - val_loss: 1.1376 - val_acc: 0.7072
Epoch 11/120
Epoch 00011: val_loss did not improve
 - 6s - loss: 0.3531 - acc: 0.8422 - val_loss: 0.7281 - val_acc: 0.7227
Epoch 12/120

Epoch 68/120
Epoch 00068: val_loss did not improve
 - 6s - loss: 0.2220 - acc: 0.9060 - val_loss: 0.1856 - val_acc: 0.9159
Epoch 69/120
Epoch 00069: val_loss did not improve
 - 6s - loss: 0.2188 - acc: 0.9037 - val_loss: 0.1882 - val_acc: 0.9190
Epoch 70/120
Epoch 00070: val_loss improved from 0.17911 to 0.17754, saving model to best_m.h5
 - 6s - loss: 0.2298 - acc: 0.8949 - val_loss: 0.1775 - val_acc: 0.9252
Epoch 71/120
Epoch 00071: val_loss did not improve
 - 6s - loss: 0.2403 - acc: 0.8954 - val_loss: 0.1814 - val_acc: 0.9190
Epoch 72/120
Epoch 00072: val_loss improved from 0.17754 to 0.17583, saving model to best_m.h5
 - 6s - loss: 0.2088 - acc: 0.9099 - val_loss: 0.1758 - val_acc: 0.9190
Epoch 73/120
Epoch 00073: val_loss improved from 0.17583 to 0.17396, saving model to best_m.h5
 - 6s - loss: 0.2310 - acc: 0.8966 - val_loss: 0.1740 - val_acc: 0.9252
Epoch 74/120
Epoch 00074: val_loss did not improve
 - 6s - loss: 0.2046 - acc: 0.9070 - val_loss: 0.1774 - val_acc: 0.9221
Epoch 7

Epoch 38/120
Epoch 00038: val_loss did not improve
 - 6s - loss: 0.2349 - acc: 0.8937 - val_loss: 0.2749 - val_acc: 0.8754
Epoch 39/120
Epoch 00039: val_loss did not improve
 - 6s - loss: 0.2349 - acc: 0.8966 - val_loss: 0.3220 - val_acc: 0.8598
Epoch 40/120
Epoch 00040: val_loss did not improve
 - 6s - loss: 0.2296 - acc: 0.8953 - val_loss: 0.2709 - val_acc: 0.8847
Epoch 41/120
Epoch 00041: val_loss did not improve
 - 6s - loss: 0.2257 - acc: 0.8930 - val_loss: 0.3098 - val_acc: 0.8785
Epoch 42/120
Epoch 00042: val_loss did not improve
 - 6s - loss: 0.2068 - acc: 0.8990 - val_loss: 0.2986 - val_acc: 0.8723
Epoch 43/120
Epoch 00043: val_loss did not improve
 - 6s - loss: 0.2367 - acc: 0.8984 - val_loss: 0.2605 - val_acc: 0.8910
Epoch 44/120
Epoch 00044: val_loss did not improve
 - 6s - loss: 0.2174 - acc: 0.9008 - val_loss: 0.2761 - val_acc: 0.8598
Epoch 45/120
Epoch 00045: val_loss did not improve
 - 6s - loss: 0.2253 - acc: 0.9091 - val_loss: 0.2607 - val_acc: 0.8723
Epoch 46/120
Epo

Epoch 47/120
Epoch 00047: val_loss did not improve
 - 6s - loss: 0.2128 - acc: 0.9109 - val_loss: 0.2460 - val_acc: 0.9034
Epoch 48/120
Epoch 00048: val_loss did not improve
 - 6s - loss: 0.1994 - acc: 0.9099 - val_loss: 0.2423 - val_acc: 0.9097
Epoch 49/120
Epoch 00049: val_loss did not improve
 - 6s - loss: 0.2267 - acc: 0.9008 - val_loss: 0.2418 - val_acc: 0.9159
Epoch 50/120
Epoch 00050: val_loss did not improve
 - 6s - loss: 0.2151 - acc: 0.9115 - val_loss: 0.2492 - val_acc: 0.8972
Epoch 51/120
Epoch 00051: val_loss did not improve
 - 6s - loss: 0.1925 - acc: 0.9125 - val_loss: 0.2422 - val_acc: 0.9159
Epoch 52/120
Epoch 00052: val_loss did not improve
 - 6s - loss: 0.2272 - acc: 0.8966 - val_loss: 0.2430 - val_acc: 0.9128
Epoch 53/120
Epoch 00053: val_loss did not improve
 - 6s - loss: 0.2242 - acc: 0.9097 - val_loss: 0.2388 - val_acc: 0.9128
Epoch 54/120
Epoch 00054: val_loss improved from 0.23793 to 0.23729, saving model to best_m.h5
 - 6s - loss: 0.2144 - acc: 0.9066 - val_los

Epoch 20/120
Epoch 00020: val_loss did not improve
 - 6s - loss: 0.2933 - acc: 0.8672 - val_loss: 0.3260 - val_acc: 0.8531
Epoch 21/120
Epoch 00021: val_loss improved from 0.29376 to 0.27080, saving model to best_m.h5
 - 6s - loss: 0.2515 - acc: 0.8828 - val_loss: 0.2708 - val_acc: 0.8844
Epoch 22/120
Epoch 00022: val_loss improved from 0.27080 to 0.26641, saving model to best_m.h5
 - 6s - loss: 0.2615 - acc: 0.8867 - val_loss: 0.2664 - val_acc: 0.8906
Epoch 23/120
Epoch 00023: val_loss did not improve
 - 6s - loss: 0.2666 - acc: 0.8649 - val_loss: 0.2903 - val_acc: 0.8812
Epoch 24/120
Epoch 00024: val_loss did not improve
 - 6s - loss: 0.2399 - acc: 0.8844 - val_loss: 0.2769 - val_acc: 0.8844
Epoch 25/120
Epoch 00025: val_loss did not improve
 - 6s - loss: 0.2714 - acc: 0.8727 - val_loss: 0.2812 - val_acc: 0.8594
Epoch 26/120
Epoch 00026: val_loss did not improve
 - 6s - loss: 0.2450 - acc: 0.8859 - val_loss: 0.2730 - val_acc: 0.8938
Epoch 27/120
Epoch 00027: val_loss did not improve


In [10]:
with open('../features/incept_aug4_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
print(log_loss(y,train_pred))
    
submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/incept_aug4_sub.csv', index=False)
# deep2 0.2110
# deep3 0.2119

0.229028814853
         id  is_iceberg
0  5941774d    0.220901
1  4023181e    0.336924
2  b20200e4    0.062305
3  e7f018bb    0.999241
4  4371c8c3    0.116947


In [13]:
from keras.layers import Input,AveragePooling2D,GlobalMaxPooling2D,GlobalAveragePooling2D,Flatten


def identity_block(input_tensor, kernel_size, filters, stage, block):
    """The identity block is the block that has no conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    """
    filters1, filters2, filters3 = filters
    bn_axis = 3
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    
    x = Conv2D(filters1, (1, 1), name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)
    
    x = Conv2D(filters2, kernel_size,
               padding='same', name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)
    
    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    x = layers.add([x, input_tensor])
    x = Activation('relu')(x)
    return x


def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):
    """A block that has a conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    Note that from stage 3, the first conv layer at main path is with strides=(2,2)
    And the shortcut should have strides=(2,2) as well
    """
    filters1, filters2, filters3 = filters

    bn_axis = 3
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    
    x = Conv2D(filters1, (1, 1), strides=strides,
               name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)
    
    x = Conv2D(filters2, kernel_size, padding='same',
               name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)
    
    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    shortcut = Conv2D(filters3, (1, 1), strides=strides,
                      name=conv_name_base + '1')(input_tensor)
    shortcut = BatchNormalization(axis=bn_axis, name=bn_name_base + '1')(shortcut)

    x = layers.add([x, shortcut])
    x = Activation('relu')(x)
    return x

def create_resnet_model():
    img_input = Input(shape=(75,75,3))
    channel_axis = 3
    bn_axis = 3
    x = Conv2D(64, (7, 7), strides=(2, 2), padding='same', name='conv1')(img_input)
    x = BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)
    x = ZeroPadding2D((1, 1))(x)

    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')

    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')
    x = ZeroPadding2D((1, 1))(x)
    
    x = Conv2D(128, 3, strides=2, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Flatten()(x)
    x = Dense(256,activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(1,activation='sigmoid')(x)
    
    return Model(img_input, x, name='inception_v3')
print('model model')


model model


In [14]:
def kfold_train(fold_cnt=3,rnd=999):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_resnet_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        early_stopping = EarlyStopping(monitor = 'val_loss', patience = 20, verbose = 0, mode= 'min')
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=120, 
                  verbose=2,
                  callbacks=[model_chk,lr_s,early_stopping]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=5)

Epoch 1/120
Epoch 00001: val_loss improved from inf to 0.73744, saving model to best_m.h5
 - 31s - loss: 1.0459 - acc: 0.5964 - val_loss: 0.7374 - val_acc: 0.5265
Epoch 2/120
Epoch 00002: val_loss improved from 0.73744 to 0.73618, saving model to best_m.h5
 - 7s - loss: 0.6184 - acc: 0.6204 - val_loss: 0.7362 - val_acc: 0.5265
Epoch 3/120
Epoch 00003: val_loss improved from 0.73618 to 0.71047, saving model to best_m.h5
 - 7s - loss: 0.6139 - acc: 0.6646 - val_loss: 0.7105 - val_acc: 0.5265
Epoch 4/120
Epoch 00004: val_loss improved from 0.71047 to 0.66811, saving model to best_m.h5
 - 7s - loss: 0.5858 - acc: 0.6836 - val_loss: 0.6681 - val_acc: 0.5265
Epoch 5/120
Epoch 00005: val_loss did not improve
 - 7s - loss: 0.5540 - acc: 0.7026 - val_loss: 0.6939 - val_acc: 0.5265
Epoch 6/120
Epoch 00006: val_loss did not improve
 - 7s - loss: 0.5324 - acc: 0.7341 - val_loss: 1.6208 - val_acc: 0.5265
Epoch 7/120
Epoch 00007: val_loss improved from 0.66811 to 0.58510, saving model to best_m.h5
 

Epoch 63/120
Epoch 00063: val_loss did not improve
 - 7s - loss: 0.2598 - acc: 0.8726 - val_loss: 0.2156 - val_acc: 0.8847
Epoch 64/120
Epoch 00064: val_loss did not improve
 - 7s - loss: 0.2743 - acc: 0.8758 - val_loss: 0.2211 - val_acc: 0.8847
Epoch 65/120
Epoch 00065: val_loss did not improve
 - 7s - loss: 0.2490 - acc: 0.8734 - val_loss: 0.2112 - val_acc: 0.9003
Epoch 66/120
Epoch 00066: val_loss did not improve
 - 7s - loss: 0.2535 - acc: 0.8873 - val_loss: 0.2211 - val_acc: 0.8941
Epoch 67/120
Epoch 00067: val_loss did not improve
 - 7s - loss: 0.2611 - acc: 0.8812 - val_loss: 0.2171 - val_acc: 0.9003
Epoch 68/120
Epoch 00068: val_loss did not improve
 - 7s - loss: 0.2589 - acc: 0.8719 - val_loss: 0.2137 - val_acc: 0.8941
Epoch 69/120
Epoch 00069: val_loss did not improve
 - 7s - loss: 0.2616 - acc: 0.8726 - val_loss: 0.2159 - val_acc: 0.8847
Epoch 70/120
Epoch 00070: val_loss did not improve
 - 7s - loss: 0.2602 - acc: 0.8740 - val_loss: 0.2177 - val_acc: 0.8879
Epoch 71/120
Epo

Epoch 51/120
Epoch 00051: val_loss improved from 0.28023 to 0.27979, saving model to best_m.h5
 - 7s - loss: 0.2612 - acc: 0.8812 - val_loss: 0.2798 - val_acc: 0.8629
Epoch 52/120
Epoch 00052: val_loss did not improve
 - 7s - loss: 0.2629 - acc: 0.8836 - val_loss: 0.3019 - val_acc: 0.8567
Epoch 53/120
Epoch 00053: val_loss did not improve
 - 7s - loss: 0.2524 - acc: 0.8812 - val_loss: 0.2902 - val_acc: 0.8567
Epoch 54/120
Epoch 00054: val_loss did not improve
 - 7s - loss: 0.2691 - acc: 0.8750 - val_loss: 0.3135 - val_acc: 0.8567
Epoch 55/120
Epoch 00055: val_loss did not improve
 - 7s - loss: 0.2315 - acc: 0.8953 - val_loss: 0.2946 - val_acc: 0.8629
Epoch 56/120
Epoch 00056: val_loss did not improve
 - 7s - loss: 0.2600 - acc: 0.8766 - val_loss: 0.3020 - val_acc: 0.8567
Epoch 57/120
Epoch 00057: val_loss did not improve
 - 7s - loss: 0.2533 - acc: 0.8867 - val_loss: 0.3001 - val_acc: 0.8536
Epoch 58/120
Epoch 00058: val_loss did not improve
 - 7s - loss: 0.2416 - acc: 0.8922 - val_los

 - 7s - loss: 0.2806 - acc: 0.8789 - val_loss: 0.2898 - val_acc: 0.8474
Epoch 42/120
Epoch 00042: val_loss did not improve
 - 7s - loss: 0.2904 - acc: 0.8699 - val_loss: 0.3033 - val_acc: 0.8380
Epoch 43/120
Epoch 00043: val_loss did not improve
 - 7s - loss: 0.2944 - acc: 0.8742 - val_loss: 0.2985 - val_acc: 0.8474
Epoch 44/120
Epoch 00044: val_loss did not improve
 - 7s - loss: 0.2887 - acc: 0.8664 - val_loss: 0.2919 - val_acc: 0.8380
Epoch 45/120
Epoch 00045: val_loss did not improve
 - 7s - loss: 0.2699 - acc: 0.8719 - val_loss: 0.3179 - val_acc: 0.8037
Epoch 46/120
Epoch 00046: val_loss improved from 0.28982 to 0.28419, saving model to best_m.h5
 - 7s - loss: 0.2670 - acc: 0.8896 - val_loss: 0.2842 - val_acc: 0.8411
Epoch 47/120
Epoch 00047: val_loss improved from 0.28419 to 0.28395, saving model to best_m.h5
 - 7s - loss: 0.2955 - acc: 0.8857 - val_loss: 0.2839 - val_acc: 0.8474
Epoch 48/120
Epoch 00048: val_loss did not improve
 - 7s - loss: 0.2518 - acc: 0.8867 - val_loss: 0.28

Epoch 104/120
Epoch 00104: val_loss did not improve
 - 7s - loss: 0.2419 - acc: 0.8977 - val_loss: 0.2643 - val_acc: 0.8692
Epoch 105/120
Epoch 00105: val_loss did not improve
 - 8s - loss: 0.2128 - acc: 0.9094 - val_loss: 0.2648 - val_acc: 0.8598
Epoch 106/120
Epoch 00106: val_loss did not improve
 - 7s - loss: 0.2227 - acc: 0.9062 - val_loss: 0.2656 - val_acc: 0.8598
Epoch 107/120
Epoch 00107: val_loss did not improve
 - 7s - loss: 0.2304 - acc: 0.9133 - val_loss: 0.2647 - val_acc: 0.8754
Epoch 108/120
Epoch 00108: val_loss did not improve
 - 7s - loss: 0.2335 - acc: 0.8964 - val_loss: 0.3035 - val_acc: 0.8567
Epoch 109/120
Epoch 00109: val_loss did not improve
 - 7s - loss: 0.2534 - acc: 0.8951 - val_loss: 0.2675 - val_acc: 0.8692
Epoch 110/120
Epoch 00110: val_loss did not improve
 - 7s - loss: 0.2234 - acc: 0.9055 - val_loss: 0.2802 - val_acc: 0.8567
Epoch 111/120
Epoch 00111: val_loss did not improve
 - 7s - loss: 0.2230 - acc: 0.8945 - val_loss: 0.2683 - val_acc: 0.8723
Epoch 11

 - 7s - loss: 0.2529 - acc: 0.8945 - val_loss: 0.2513 - val_acc: 0.8879
Epoch 50/120
Epoch 00050: val_loss did not improve
 - 7s - loss: 0.2453 - acc: 0.8797 - val_loss: 0.2572 - val_acc: 0.8941
Epoch 51/120
Epoch 00051: val_loss did not improve
 - 7s - loss: 0.2363 - acc: 0.8865 - val_loss: 0.2589 - val_acc: 0.9003
Epoch 52/120
Epoch 00052: val_loss did not improve
 - 7s - loss: 0.2546 - acc: 0.8724 - val_loss: 0.2534 - val_acc: 0.8910
Epoch 53/120
Epoch 00053: val_loss improved from 0.25129 to 0.24557, saving model to best_m.h5
 - 7s - loss: 0.2444 - acc: 0.8914 - val_loss: 0.2456 - val_acc: 0.8847
Epoch 54/120
Epoch 00054: val_loss improved from 0.24557 to 0.24397, saving model to best_m.h5
 - 7s - loss: 0.2489 - acc: 0.8834 - val_loss: 0.2440 - val_acc: 0.8972
Epoch 55/120
Epoch 00055: val_loss did not improve
 - 7s - loss: 0.2273 - acc: 0.8992 - val_loss: 0.2485 - val_acc: 0.8910
Epoch 56/120
Epoch 00056: val_loss did not improve
 - 7s - loss: 0.2583 - acc: 0.8761 - val_loss: 0.25

Epoch 37/120
Epoch 00037: val_loss did not improve
 - 7s - loss: 0.2829 - acc: 0.8695 - val_loss: 0.2610 - val_acc: 0.8625
Epoch 38/120
Epoch 00038: val_loss improved from 0.25652 to 0.24370, saving model to best_m.h5
 - 7s - loss: 0.2880 - acc: 0.8664 - val_loss: 0.2437 - val_acc: 0.8812
Epoch 39/120
Epoch 00039: val_loss did not improve
 - 7s - loss: 0.2963 - acc: 0.8688 - val_loss: 0.3788 - val_acc: 0.8094
Epoch 40/120
Epoch 00040: val_loss did not improve
 - 7s - loss: 0.2802 - acc: 0.8633 - val_loss: 0.3128 - val_acc: 0.8625
Epoch 41/120
Epoch 00041: val_loss did not improve
 - 7s - loss: 0.2920 - acc: 0.8711 - val_loss: 0.3007 - val_acc: 0.8750
Epoch 42/120
Epoch 00042: val_loss did not improve
 - 7s - loss: 0.2844 - acc: 0.8640 - val_loss: 0.2828 - val_acc: 0.8750
Epoch 43/120
Epoch 00043: val_loss did not improve
 - 7s - loss: 0.2608 - acc: 0.8812 - val_loss: 0.2700 - val_acc: 0.8656
Epoch 44/120
Epoch 00044: val_loss improved from 0.24370 to 0.23567, saving model to best_m.h5


Epoch 99/120
Epoch 00099: val_loss did not improve
 - 7s - loss: 0.2406 - acc: 0.8906 - val_loss: 0.2395 - val_acc: 0.8938
Epoch 100/120
Epoch 00100: val_loss did not improve
 - 7s - loss: 0.2412 - acc: 0.8898 - val_loss: 0.2224 - val_acc: 0.9094
Epoch 101/120
Epoch 00101: val_loss did not improve
 - 7s - loss: 0.2323 - acc: 0.8977 - val_loss: 0.2301 - val_acc: 0.8969
Epoch 102/120
Epoch 00102: val_loss did not improve
 - 7s - loss: 0.2398 - acc: 0.8914 - val_loss: 0.2237 - val_acc: 0.9094
Epoch 103/120
Epoch 00103: val_loss did not improve
 - 7s - loss: 0.2472 - acc: 0.8844 - val_loss: 0.2589 - val_acc: 0.8875
Epoch 104/120
Epoch 00104: val_loss did not improve
 - 7s - loss: 0.2268 - acc: 0.8930 - val_loss: 0.2312 - val_acc: 0.8875
Epoch 105/120
Epoch 00105: val_loss did not improve
 - 7s - loss: 0.2421 - acc: 0.8898 - val_loss: 0.2234 - val_acc: 0.9094
Epoch 106/120
Epoch 00106: val_loss improved from 0.21489 to 0.21333, saving model to best_m.h5
 - 7s - loss: 0.2329 - acc: 0.8992 - 

In [15]:
with open('../features/resnet_aug4_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
print(log_loss(y,train_pred))
    
submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/resnet_aug4_sub.csv', index=False)
# deep2 0.227
# deep3 0.223

0.242099828676
         id  is_iceberg
0  5941774d    0.479608
1  4023181e    0.410470
2  b20200e4    0.112072
3  e7f018bb    0.989842
4  4371c8c3    0.206648
